In [77]:
using LazySets: isapproxzero, isapprox
function _four_points_2dv2!(points::AbstractVector{<:AbstractVector{N}}) where {N<:Real}
    A, B, C, D = points[1], points[2], points[3], points[4]
    tri_ABC = (A[2] - B[2]) * C[1] + (B[1] - A[1]) * C[2] + (A[1] * B[2] - B[1] * A[2])
    tri_ABD = (A[2] - B[2]) * D[1] + (B[1] - A[1]) * D[2] + (A[1] * B[2] - B[1] * A[2])
    tri_BCD = (B[2] - C[2]) * D[1] + (C[1] - B[1]) * D[2] + (B[1] * C[2] - C[1] * B[2])
    tri_CAD = (C[2] - A[2]) * D[1] + (A[1] - C[1]) * D[2] + (C[1] * A[2] - A[1] * C[2])
    key = 0
    if tri_ABC > zero(N)
        key = key + 1000
    end
    if tri_ABD > zero(N)
        key = key + 100
    end
    if tri_BCD > zero(N)
        key = key + 10
    end
    if tri_CAD > zero(N)
        key = key + 1
    end
    
    function collinear_case(A, B, C, D)
        # A, B and C collinear, D is the extra point
        if isapprox(A[1], B[1]) && isapprox(B[1], C[1]) && isapprox(C[1], A[1])
            # points are approximately equal in their first component
            if isapprox(A[2], B[2]) && isapprox(B[2], C[2]) && isapprox(C[2], A[2])
                # the three points are approximately equal
                points[1], points[2] = A, D
                pop!(points)
                pop!(points)
                return _two_points_2d!(points)
            else
                # assign the points with max and min value in their second component to the
                # firsts points and the extra point to the third place, then pop the point that was in the middle
                points[1], points[2], points[3] = points[argmin([A[2], B[2], C[2]])], points[argmax([A[2], B[2], C[2]])], D
                pop!(points)
            end
        else
            # assign the points with max and min value in their first component to the
            # firsts points and the extra point to the third place, then pop the point that was in the middle
            points[1], points[2], points[3] = points[argmin([A[1], B[1], C[1]])], points[argmax([A[1], B[1], C[1]])], D
            pop!(points)
        end
        return _three_points_2d!(points)
    end
    
    if isapproxzero(tri_ABC)
        return collinear_case(A, B, C, D)
    end
    if isapproxzero(tri_ABD)
        return collinear_case(A, B, D, C)
    end
    if isapproxzero(tri_BCD)
        return collinear_case(B, C, D, A)
    end
    if isapproxzero(tri_CAD)
        return collinear_case(C, A, D, B)
    end
    
    # ABC  ABD  BCD  CAD  hull
    # ------------------------
    #  +    +    +    +   ABC
    #  +    +    +    -   ABCD
    #  +    +    -    +   ABDC
    #  +    +    -    -   ABD
    #  +    -    +    +   ADBC
    #  +    -    +    -   BCD
    #  +    -    -    +   CAD
    #  +    -    -    -   [should not happen]
    #  -    +    +    +   [should not happen]
    #  -    +    +    -   ACD
    #  -    +    -    +   DCB
    #  -    +    -    -   DBCA
    #  -    -    +    +   ADB
    #  -    -    +    -   ACDB
    #  -    -    -    +   ADCB
    #  -    -    -    -   ACB
    
    #  +    +    +    +   ABC
    function _four_points_2d_helper!(key::Val{1111})
        points[1], points[2], points[3] = A, B, C
        pop!(points)
    end
    #  +    +    +    -   ABCD
    function _four_points_2d_helper!(key::Val{1110})
        points[1], points[2], points[3], points[4] = A, B, C, D
    end
    #  +    +    -    +   ABDC
    function _four_points_2d_helper!(key::Val{1101})
        points[1], points[2], points[3], points[4] = A, B, D, C
    end
    #  +    +    -    -   ABD
    function _four_points_2d_helper!(key::Val{1100})
        points[1], points[2], points[3] = A, B, D
        pop!(points)
    end
    #  +    -    +    +   ADBC
    function _four_points_2d_helper!(key::Val{1011})
        points[1], points[2], points[3], points[4] = A, D, B, C
    end
    #  +    -    +    -   BCD
    function _four_points_2d_helper!(key::Val{1010})
        points[1], points[2], points[3] = B, C, D
        pop!(points)
    end
    #  +    -    -    +   CAD
    function _four_points_2d_helper!(key::Val{1001})
        points[1], points[2], points[3] = C, A, D
        pop!(points)
    end
    #  +    -    -    -   [should not happen] fallback
    function _four_points_2d_helper!(key::Val{1000})
        @assert false "unexpected case in convex_hull"
    end
    #  -    +    +    +   [should not happen] fallback
    function _four_points_2d_helper!(key::Val{0111})
        @assert false "unexpected case in convex_hull"
    end
    #  -    +    +    -   ACD
    function _four_points_2d_helper!(key::Val{0110})
        points[1], points[2], points[3] = A, C, D
        pop!(points)
    end
    #  -    +    -    +   DCB
    function _four_points_2d_helper!(key::Val{0101})
        points[1], points[2], points[3] = D, C, B
        pop!(points)
    end
    #  -    +    -    -   DBCA
    function _four_points_2d_helper!(key::Val{0100})
        points[1], points[2], points[3], points[4] = A, B, C, A
    end
    #  -    -    +    +   ADB
    function _four_points_2d_helper!(key::Val{0011})
        points[1], points[2], points[3] = A, D, B
        pop!(points)
    end
    #  -    -    +    -   ACDB
    function _four_points_2d_helper!(key::Val{0010})
        points[1], points[2], points[3], points[4] = A, C, D, B
    end
    #  -    -    -    +   ADCB
    function _four_points_2d_helper!(key::Val{0001})
        points[1], points[2], points[3], points[4] = A, D, C, B
    end
    #  -    -    -    -   ACB
    function _four_points_2d_helper!(key::Val{0000})
        points[1], points[2], points[3] = A, C, B
        pop!(points)
    end
    _four_points_2d_helper!(Val(key))
    return points
end

_four_points_2dv2! (generic function with 1 method)

In [76]:
function _four_points_2d!(points::AbstractVector{<:AbstractVector{N}}) where {N<:Real}
    A, B, C, D = points[1], points[2], points[3], points[4]
    tri_ABC = (A[2] - B[2]) * C[1] + (B[1] - A[1]) * C[2] + (A[1] * B[2] - B[1] * A[2])
    tri_ABD = (A[2] - B[2]) * D[1] + (B[1] - A[1]) * D[2] + (A[1] * B[2] - B[1] * A[2])
    tri_BCD = (B[2] - C[2]) * D[1] + (C[1] - B[1]) * D[2] + (B[1] * C[2] - C[1] * B[2])
    tri_CAD = (C[2] - A[2]) * D[1] + (A[1] - C[1]) * D[2] + (C[1] * A[2] - A[1] * C[2])
    key = 0
    if tri_ABC > zero(N)
        key = key + 1000
    end
    if tri_ABD > zero(N)
        key = key + 100
    end
    if tri_BCD > zero(N)
        key = key + 10
    end
    if tri_CAD > zero(N)
        key = key + 1
    end
    
    function collinear_case(A, B, C, D)
        # A, B and C collinear, D is the extra point
        if isapprox(A[1], B[1]) && isapprox(B[1], C[1]) && isapprox(C[1], A[1])
            # points are approximately equal in their first component
            if isapprox(A[2], B[2]) && isapprox(B[2], C[2]) && isapprox(C[2], A[2])
                # the three points are approximately equal
                points[1], points[2] = A, D
                pop!(points)
                pop!(points)
                return _two_points_2d!(points)
            else
                # assign the points with max and min value in their second component to the
                # firsts points and the extra point to the third place, then pop the point that was in the middle
                points[1], points[2], points[3] = points[argmin([A[2], B[2], C[2]])], points[argmax([A[2], B[2], C[2]])], D
                pop!(points)
            end
        else
            # assign the points with max and min value in their first component to the
            # firsts points and the extra point to the third place, then pop the point that was in the middle
            points[1], points[2], points[3] = points[argmin([A[1], B[1], C[1]])], points[argmax([A[1], B[1], C[1]])], D
            pop!(points)
        end
        return _three_points_2d!(points)
    end
    
    if isapproxzero(tri_ABC)
        return collinear_case(A, B, C, D)
    end
    if isapproxzero(tri_ABD)
        return collinear_case(A, B, D, C)
    end
    if isapproxzero(tri_BCD)
        return collinear_case(B, C, D, A)
    end
    if isapproxzero(tri_CAD)
        return collinear_case(C, A, D, B)
    end
    
    # ABC  ABD  BCD  CAD  hull
    # ------------------------
    #  +    +    +    +   ABC
    #  +    +    +    -   ABCD
    #  +    +    -    +   ABDC
    #  +    +    -    -   ABD
    #  +    -    +    +   ADBC
    #  +    -    +    -   BCD
    #  +    -    -    +   CAD
    #  +    -    -    -   [should not happen]
    #  -    +    +    +   [should not happen]
    #  -    +    +    -   ACD
    #  -    +    -    +   DCB
    #  -    +    -    -   DBCA
    #  -    -    +    +   ADB
    #  -    -    +    -   ACDB
    #  -    -    -    +   ADCB
    #  -    -    -    -   ACB
    if key == 1111
        points[1], points[2], points[3] = A, B, C # +    +    +    +   ABC
        pop!(points)
    elseif key == 1110
        points[1], points[2], points[3], points[4] = A, B, C, D # +    +    +    -   ABCD
    elseif key == 1101
        points[1], points[2], points[3], points[4] = A, B, D, C #  +    +    -    +   ABDC
    elseif key == 1100
        points[1], points[2], points[3] = A, B, D #  +    +    -    -   ABD
        pop!(points)
    elseif key == 1011
        points[1], points[2], points[3], points[4] = A, D, B, C #  +    -    +    +   ADBC
    elseif key == 1010
        points[1], points[2], points[3] = B, C, D #  +    -    +    -   BCD
        pop!(points)
    elseif key == 1001
        points[1], points[2], points[3] = C, A, D #  +    -    -    +    CAD
        pop!(points)
    elseif key == 1000
        @assert false "unexpected case in convex_hull" #  +    -    -    -   [should not happen]
    elseif key == 0111
        @assert false "unexpected case in convex_hull" #  -    +    +    +   [should not happen]
    elseif key == 0110
        points[1], points[2], points[3] = A, C, D #  -    +    +    -   ACD
        pop!(points)
    elseif key == 0101
        points[1], points[2], points[3] = D, C, B #  -    +    -    +   DCB
        pop!(points)
    elseif key == 0100
        points[1], points[2], points[3], points[4] = D, B, C, A #  -    +    -    -   DBCA
    elseif key == 0011
        points[1], points[2], points[3] = A, D, B #  -    -    +    +   ADB
        pop!(points)
    elseif key == 0010
        points[1], points[2], points[3], points[4] = A, C, D, B #  -    -    +    -   ACDB
    elseif key == 0001
        points[1], points[2], points[3], points[4] = A, D, C, B #  -    -    -    +   ADCB
    elseif key == 0000
        points[1], points[2], points[3] = A, C, B #  -    -    -    -   ACB
        pop!(points)
    end
    return points
end

_four_points_2d! (generic function with 1 method)

In [89]:
using BenchmarkTools
N = Float64
# four vertex case in 2 dimentions 
A = N[1, 0] 
B = N[1, 1] 
C = N[-1, 1] 
D = N[-1, 0] 
expr = [A, B, C, D]
points = [C, B, D, A]
@btime _four_points_2d!(copy($points))

  90.543 ns (2 allocations: 128 bytes)


4-element Array{Array{Float64,1},1}:
 [-1.0, 1.0]
 [-1.0, 0.0]
 [1.0, 0.0] 
 [1.0, 1.0] 

In [88]:
@btime _four_points_2dv2!(copy($points))

  5.935 μs (4 allocations: 224 bytes)


4-element Array{Array{Float64,1},1}:
 [-1.0, 1.0]
 [-1.0, 0.0]
 [1.0, 0.0] 
 [1.0, 1.0] 